In [11]:
#all import statements
import os
import pandas as pd
import torch 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np



In [12]:

model_directory = 'saved_models'
model_filename = 'linear_regression_model.pth'
model_file_path = os.path.join(model_directory, model_filename)

# Ensure the directory exists, create it if it doesn't
os.makedirs(model_directory, exist_ok=True)
# Function to load or initialize the model
def load_or_initialize_model(X_train, y_train):

    # Load the model if it exists, otherwise initialize a new model

    if 0 and os.path.exists(model_file_path):
        print('Model exist so using it')
        model = torch.load(model_file_path)
    else:
        print('Model does not exist so initialising from scratch')
        input_size = X_train.shape[1]
        model = nn.Linear(input_size, 1).double()
        torch.save(model, model_file_path)
    return model

In [13]:
#function to train the model
def modeltrain(X_train, y_train, l2_lambda=0.01):  # Specify the regularization strength (lambda)

    input_size = X_train.shape[1]
    model = nn.Linear(input_size, 1).double()  # Double data type for weight tensor

    # Define loss function and optimizer with L2 regularization
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=l2_lambda)  # Adding weight decay for L2 regularization

    # Train the model (rest of the code remains the same)

    
    num_epochs = 1000
    

    for epoch in range(num_epochs):
        # Forward pass
        outputs = model(X_train.double())  # Ensure input data type matches model's weight data type
        loss = criterion(outputs, y_train.view(-1, 1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    print('Training Done')
    return model


In [21]:
#function to test the model
predictions=''
y_test=''
def modeltest(X_test,y_1,model):
    global predictions
    global y_test
    y_test=y_1
    print("Testing starting")
    # Assuming 'model' is your trained linear regression model
    # print(model)
    with torch.no_grad():
        predictions = model(X_test)


    print('testing over now accuracy analysis')
    # print(predictions)
    correct_pred=0
    for i in range(len(predictions)):
        if(abs(predictions[i]-y_test[i])<0.1):
            correct_pred+=1
    mean_accuracy=(correct_pred/len(predictions))*100
    print(f'Percentage Accuracy: {mean_accuracy:.2f}%')

    # for (int)
    # Calculate the mean of the actual target values
    mean_actual = torch.mean(y_test)

    # Calculate the total sum of squares (TSS)
    tss = torch.sum((y_test - mean_actual)**2)

    # Calculate the residual sum of squares (RSS)
    rss = torch.sum((y_test - predictions)**2)

    # Calculate R-squared (R2)
    r2 = 1 - (rss / tss)
    print("r2 score is ",r2)

    # Calculate MAE and MSE
    # mae = mean_absolute_error(y_test, predictions)
    # mse = mean_squared_error(y_test, predictions)

    # print(f'MAE: {mae}')
    # print(f'MSE: {mse}')

In [22]:

def create_plot(X,y):
    # Replace 'X1' with the name of the feature you want to plot.
    # n=int(input("Enter the feature for which you want the plot, enter any value b/w 0-2:"))
    # n1=int(input("Enter the feature for which you want the plot, enter any value b/w 0-2:"))
    # n2=int(input("Enter the feature for which you want the plot, enter any value b/w 0-2:"))

    feature_to_plot = X[:,0] 
    feature_to_plot1 = X[:,1] 
    feature_to_plot2 = X[:,2] 
    
    


    # Create a scatter plotc.ear
    # plt.scatter(feature_to_plot, y, alpha=0.5)
    # plt.show()
    plt.scatter(feature_to_plot1, y, alpha=0.5)
    # plt.show()
    plt.scatter(feature_to_plot2, y, alpha=0.5)

    # plt.plot(feature_to_plot, y, alpha=0.5)  #line plot
    plt.xlabel('Feature No:')  # Replace with the appropriate feature name
    plt.ylabel('y')
    plt.title('Line Plot of Feature vs. y')
    plt.show()

In [23]:





#Data  proccessing and normalisation tried

df = pd.read_csv('Q3_complex_linear_regression_dataset.csv')
#print(df.head())



# Separate the target variable 'y' from the input features.
X = df.drop(columns=['y'])
y = df['y']

# Handle categorical variables using one-hot encoding.
X = pd.get_dummies(X, columns=['X3'], prefix=['X3'])

for column in X.columns:
    try:
        X[column] = X[column].astype(float)
    except ValueError:
        print(f"Column '{column}' cannot be converted to float.")



# Convert the DataFrame to a PyTorch tensor.
X = torch.tensor(X.values, dtype=torch.float64)
list_X1=[]
listd=[]
for i in range(len(X)):
    list_X1.append(X[i][0].item())
    listd.append([X[i][0].item(),X[i][1].item(),X[i][2].item()])


# print(list_X1)
list_X1=torch.tensor(list_X1)
mean_X1 = torch.mean(list_X1).item()
std_X1 = torch.std(list_X1).item()
# print(mean_X1.item(),std_X1.item())

for i in range(len(X)):
    listd[i][0]=(listd[i][0]-mean_X1)/std_X1
listd=torch.tensor(listd)
# print("X befor normalisation",X)
# X=listd
# print('X after normalsation',X)

# Convert the target variable to a PyTorch tensor.
y = torch.tensor(y.values, dtype=torch.float64)
size=len(X)
train_ratio=0.7
train_size=int(size*train_ratio)
test_size=size-train_size


#Splitting the data in training and testing data 


#Training data
X_train=X[:train_size]
y_train=y[:train_size]

#Testing data
X_test=X[train_size:]
y_test=y[train_size:]

In [24]:
#Relation of input variables with each other (EDA)
input1=[]
input2=[]
input3=[]

for i in range(len(X_train)):
    c=X_train[i][0]

    input1.append(c.item())
    c=X_train[i][1]

    input2.append(c.item())
    c=X_train[i][2]
    input3.append(c.item())

covariance_matrix = np.cov(input1, input2)

# Extract the covariance between the two variables from the covariance matrix
covariance = covariance_matrix[0, 1]

# print("Covariance between input1 and input2:", covariance)

covariance_matrix1 = np.cov(input1, input3)
covariance_matrix2 = np.cov(input3, input2)
covariance1 = covariance_matrix1[0, 1]
covariance2 = covariance_matrix2[0, 1]
print("Covariance between input1 and input2:", covariance)
print("Covariance between input2 and input3:", covariance1)
print("Covariance between input3 and input2:", covariance2)



Covariance between input1 and input2: 0.12740488844015435
Covariance between input2 and input3: -0.025787961695156975
Covariance between input3 and input2: 0.012576896787423134


In [25]:
# create_plot(X_train,y_train)
# model = load_or_initialize_model(X_train, y_train)
# model=modeltrain(X_train,y_train)
model = load_or_initialize_model(X_train,y_train)
model = modeltrain(X_train, y_train, l2_lambda=0.01)  # You can adjust the regularization st
modeltest(X_test,y_test,model)

Model does not exist so initialising from scratch
Epoch [100/1000], Loss: 4.141375067378297
Epoch [200/1000], Loss: 4.003931521926671
Epoch [300/1000], Loss: 3.942616421426183
Epoch [400/1000], Loss: 3.9140671783272083
Epoch [500/1000], Loss: 3.900372975874568
Epoch [600/1000], Loss: 3.8936290703821776
Epoch [700/1000], Loss: 3.890214256853382
Epoch [800/1000], Loss: 3.8884301308890565
Epoch [900/1000], Loss: 3.887464927048166
Epoch [1000/1000], Loss: 3.8869231149382335
Training Done
Testing starting
testing over now accuracy analysis
Percentage Accuracy: 4.44%
r2 score is  tensor(-174.1296, dtype=torch.float64)


In [47]:
print(type(predictions[0]))
len(predictions)
n=len(y_test)
k=0


# x=predictions
x = np.array([scalar.item() for scalar in predictions])
y=y_test
y = np.array([scalar.item() for scalar in y_test])
print(x[89])

print(y[89])

print(x[:10])
print(y[:10])

x=torch.tensor(x)
y=torch.tensor(y)
mean_actual = torch.mean(y)

# Calculate the total sum of squares (TSS)
tss = torch.sum((y - mean_actual)**2)

# Calculate the residual sum of squares (RSS)
rss = torch.sum((y - x)**2)

# Calculate R-squared (R2)
r2 = 1 - (rss / tss)
print("r2 score is ",r2)

# flag=0
# num=int(input("\nenter number of values : \t"))
# while (num!=0):
#     print("prediction | actual")
#     for i in range (num):
#         print(round(x[k],3),end='\t')
#         print(round(y[k],3))
#         k+=1
#         if (k==n):
#             flag=1
#             break
        
#     if (flag==1):
#         print('ended')
#         break
    # num=int(input("\nenter number of values : \t"))
    





<class 'torch.Tensor'>
31.216983758626668
32.266406881190946
[32.11499355  8.5740621  19.99559605  7.90661896  0.52053115 22.97444103
  9.63221423 13.37936278 17.79834864 25.18626156]
[34.22944283 10.29024283 20.90358031  6.53283796 -1.13345215 22.74013029
 12.53363921 12.05232975 14.18387843 26.33137223]
r2 score is  tensor(0.9605, dtype=torch.float64)
